In [ ]:
#Eólica Offshore 875HPA

import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            'u_component_of_wind', 'v_component_of_wind',
        ],
        'pressure_level': '875',
        'year': '2022',
         'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            -27, -45, -20, -35,  # Coordenadas da Bacia de Campos e um pouco mais pegando a costa brasileira
        ],
    },
    'Era5_2022_875HPA_CAMPOS.nc')
import xarray as xr

titulo = "Marcação das Plataformas na Bacia de Campos Desativadas"

titulo_html = """<h1 align="center" style="font-family: verdana; font-size:16px; font-weight:bold; text-transform: uppercase;">{}</h1>""".format(titulo)


2024-02-19 21:57:12,984 INFO Welcome to the CDS
2024-02-19 21:57:12,984 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-02-19 21:57:13,305 INFO Request is queued


In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

# Carrega o arquivo NetCDF com os dados
ds = xr.open_dataset('Era5_2022_875HPA_CAMPOS.nc', engine='netcdf4')

# Calcula a velocidade do vento (wspd) a partir das componentes u e v
u = ds['u']
v = ds['v']
wspd = (u**2 + v**2)**0.5

# Calcula a média da velocidade do vento para cada ponto de grade ao longo dos horários e dias, agrupando por estação do ano
wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

# Adiciona marcadores para as localizações das plataformas de petróleo e gás
platforms = {
    'NAMORADO 2 (PNA-2)' :(-22.45073, -40.41175),
    'PETROBRAS 26 (P-26)' :(-22.4684, -40.02869),
    'PETROBRAS 32 (P-32)' :(-22.2051, -40.1431),
    'PETROBRAS 37 (P-37)' :(-22.4868, -40.09779),
    'PETROBRAS IX' :(-22.57358, -40.82192),
    'PETROBRAS XIX' :(-22.3927, -40.05438),
    'PETROBRAS XXXIII' :(-22.37, -40.0267),
    'VERMELHO 1 (PVM-1)' :(-22.16065, -40.27872),
    'VERMELHO 2 (PVM-2)' :(-22.17535, -40.29147),
}

# Cria um DataFrame para armazenar os resultados
result_df = pd.DataFrame(index=platforms.keys(), columns=['DJF', 'MAM', 'JJA', 'SON'])

# Loop sobre as estações do ano
for season in ['DJF', 'MAM', 'JJA', 'SON']:
    for platform, (lat, lon) in platforms.items():
        # Obtém o valor da média da velocidade do vento para a plataforma e estação do ano específicos
        value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()
        result_df.at[platform, season] = value

# Exibe o DataFrame
print(result_df)

# Fecha o arquivo NetCDF
ds.close()

                          DJF       MAM       JJA       SON
NAMORADO 2 (PNA-2)   7.151781  5.910459  6.903085  6.767473
PETROBRAS 26 (P-26)  7.504502  5.931946   6.95721  7.151103
PETROBRAS 32 (P-32)  7.242642  5.820916  6.765968  6.843705
PETROBRAS 37 (P-37)  7.504502  5.931946   6.95721  7.151103
PETROBRAS IX         6.923608  5.816296  6.808736  6.611757
PETROBRAS XIX        7.504502  5.931946   6.95721  7.151103
PETROBRAS XXXIII     7.407397  5.838326  6.799466  7.044631
VERMELHO 1 (PVM-1)   7.242642  5.820916  6.765968  6.843705
VERMELHO 2 (PVM-2)   7.242642  5.820916  6.765968  6.843705
